In [1]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)


In [2]:
sys.path

['',
 '/home/kai/anaconda3/lib/python36.zip',
 '/home/kai/anaconda3/lib/python3.6',
 '/home/kai/anaconda3/lib/python3.6/lib-dynload',
 '/home/kai/anaconda3/lib/python3.6/site-packages',
 '/home/kai/anaconda3/lib/python3.6/site-packages/xgboost-0.7-py3.6.egg',
 '/home/kai/anaconda3/lib/python3.6/site-packages/duckling-1.8.0-py3.6.egg',
 '/home/kai/anaconda3/lib/python3.6/site-packages/JPype1-0.6.3-py3.6-linux-x86_64.egg',
 '/home/kai/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/kai/.ipython',
 '/home/kai/data/shiyi/AlphaBoosting/automl']

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels 

In [6]:
# params
#param = {'trainLen': self.train_len, 'splitCol': 'a', 'col': self.label, 'coefficient': 10, 'n': 2, 'fillna': 22}
# feature_engineering_dict.append({'params': params, 'function': 'count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'unique_count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'cumulative_count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'reverse_cumulative_count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'variance', 'feature_cols': ['a','n']})
# feature_engineering_dict.append({'params': params, 'function': 'count_std_over_mean', 'feature_cols': ['a','b']})


# feature_engineering_dict.append({'params': params, 'function': 'time_to_n_next', 'feature_cols': ['a','t']})
# feature_engineering_dict.append({'params': params, 'function': 'count_in_previous_n_time_unit', 'feature_cols': ['a','t']})
# feature_engineering_dict.append({'params': params, 'function': 'count_in_next_n_time_unit', 'feature_cols': ['a','t']})

# feature_engineering_dict.append({'params': params, 'function': 'woe', 'feature_cols': ['a']})
# feature_engineering_dict.append({'params': params, 'function': 'chi_square', 'feature_cols': ['a']})
# feature_engineering_dict.append({'params': params, 'function': 'mean', 'feature_cols': ['a']})

In [7]:
categorical_features = ['user', 'movie', 'age', 'gender']#, 'occupation', 'zip', 'released']

In [8]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list):
    features_to_gen = []
    for function in function_list:
        for i in range(2, len(categorical_features)+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

features_to_gen = get_features_to_gen([fe.count])#, fe.unique_count, fe.cumulative_count,
                                     #fe.reverse_cumulative_count, fe.variance, fe.count_std_over_mean])
# features_to_gen = get_features_to_gen([])#fe.count])

features_to_gen

[{'feature_cols': ['user', 'movie'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['user', 'age'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['user', 'gender'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['movie', 'age'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['movie', 'gender'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['age', 'gender'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['user', 'movie', 'age'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['user', 'movie', 'gender'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['user', 'age', 'gend

In [9]:
len(features_to_gen)

11

In [10]:
project_path = '/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project2/' 

In [11]:
from automl_app import logger_config

logger_config.config(project_path+'log_project2.log')

In [12]:
def lgb_gs_params_gen():
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    params = {    
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 20,
        'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
        'num_leaves': np.random.choice([15,31,61,127]),
        'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
        'feature_fraction': np.random.randint(4,11)/10,
        'feature_fraction_seed': seed,
        'early_stopping_round':70,
        'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
        'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
        'bagging_seed': seed,
        'scale_pos_weight': 2,
        'metric' : 'auc' 
    }
    return params, seed

In [13]:
lgb_gs_params_gen()

({'bagging_fraction': 0.6,
  'bagging_freq': 1,
  'bagging_seed': 27123012,
  'boosting': 'gbdt',
  'early_stopping_round': 70,
  'feature_fraction': 1.0,
  'feature_fraction_seed': 27123012,
  'learning_rate': 0.029999999999999999,
  'max_depth': -1,
  'metric': 'auc',
  'min_data_in_leaf': 40,
  'num_leaves': 31,
  'num_rounds': 20,
  'num_threads': 8,
  'objective': 'binary',
  'scale_pos_weight': 2},
 27123012)

In [ ]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, lgb_gs_params_gen)

> /home/kai/data/shiyi/AlphaBoosting/automl/automl_app/app.py(129)_read_data()
-> self.df = pd.concat([self.train, self.test], ignore_index=True)


In [ ]:
getattr(np.random, 'randint')

In [ ]:
getattr(fe, 'count')